In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import researchpy as rp
import seaborn as sns
import statsmodels.api as sm
from statsmodels.formula.api import ols
import statsmodels.stats.multicomp

from jupyterthemes import jtplot
jtplot.style(theme="grade3")

# Load Data

In [2]:
datasize = ["256", "512", "128K", "256K"]

In [3]:
realTime_Unloaded_FreqDisable = np.zeros([600, len(datasize)])
for i in range(len(datasize)):
    realTime_Unloaded_FreqDisable[:, i] = np.loadtxt("./q5/Real-Time_Unloaded_FreqDisable/"+"transport_time_"+datasize[i]+"byte.txt", dtype=np.float64) * 1000 # convert "s" to "ms"

realTime_Unloaded_FreqDisable = realTime_Unloaded_FreqDisable[30:, :]
print("realTime_Unloaded_FreqDisable: ", realTime_Unloaded_FreqDisable.shape)

realTime_Unloaded_FreqDisable:  (570, 4)


In [4]:
realTime_Loaded_FreqDisable = np.zeros([600, len(datasize)])
for i in range(len(datasize)):
    realTime_Loaded_FreqDisable[:, i] = np.loadtxt("./q5/Real-Time_Loaded_FreqDisable/"+"transport_time_"+datasize[i]+"byte.txt", dtype=np.float64) * 1000 # convert "s" to "ms"
    
realTime_Loaded_FreqDisable = realTime_Loaded_FreqDisable[30:, :]
print("realTime_Loaded_FreqDisable: ", realTime_Loaded_FreqDisable.shape)

realTime_Loaded_FreqDisable:  (570, 4)


In [5]:
nonRealTime_Unloaded_FreqDisable = np.zeros([600, len(datasize)])
for i in range(len(datasize)):
    nonRealTime_Unloaded_FreqDisable[:, i] = np.loadtxt("./q5/NonReal-Time_Unloaded_FreqDisable/"+"transport_time_"+datasize[i]+"byte.txt", dtype=np.float64) * 1000 # convert "s" to "ms"
    
nonRealTime_Unloaded_FreqDisable = nonRealTime_Unloaded_FreqDisable[30:, :]
print("nonRealTime_Unloaded_FreqDisable: ", nonRealTime_Unloaded_FreqDisable.shape)

nonRealTime_Unloaded_FreqDisable:  (570, 4)


In [6]:
nonRealTime_Loaded_FreqDisable = np.zeros([600, len(datasize)])
for i in range(len(datasize)):
    nonRealTime_Loaded_FreqDisable[:, i] = np.loadtxt("./q5/NonReal-Time_Loaded_FreqDisable/"+"transport_time_"+datasize[i]+"byte.txt", dtype=np.float64) * 1000 # convert "s" to "ms"
    
nonRealTime_Loaded_FreqDisable = nonRealTime_Loaded_FreqDisable[30:, :]
print("nonRealTime_Loaded_FreqDisable: ", nonRealTime_Loaded_FreqDisable.shape)

nonRealTime_Loaded_FreqDisable:  (570, 4)
